# Autoencoders

<img src="https://blog.keras.io/img/ae/autoencoder_schema.jpg"/>


In [1]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


## Prepare the data

In [2]:
from keras.datasets import mnist
import numpy as np

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

## Open Tensorboard to see live training

```sh
$ tensorboard --logdir=/tmp/autoencoder
```

In [5]:
# tensorboard --logdir=/tmp/autoencoder 

In [6]:
%%html

<iframe src="http://localhost:6006" style="width:100%; height: 1000px"/>

In [7]:
from keras.callbacks import TensorBoard

autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 59s 982us/step - loss: 0.2181 - val_loss: 0.1692
Epoch 2/50
60000/60000 [==============================] - 56s 927us/step - loss: 0.1564 - val_loss: 0.1464
Epoch 3/50
60000/60000 [==============================] - 57s 944us/step - loss: 0.1435 - val_loss: 0.1380
Epoch 4/50
25984/60000 [===========>..................] - ETA: 34s - loss: 0.1375

KeyboardInterrupt: 

In [8]:
autoencoder.save("data/autoencoder_mnist.h5")
autoencoder.save_weights("data/autoencoder_weights_mnist.h5")

## Refs

- https://blog.keras.io/building-autoencoders-in-keras.html